# Azure AI Evaluation client library for Python

Use Azure AI Evaluation SDK to assess the performance of your generative AI applications. Generative AI application generations are quantitatively measured with mathematical based metrics, AI-assisted quality and safety metrics. Metrics are defined as evaluators. Built-in or custom evaluators can provide comprehensive insights into the application's capabilities and limitations.

Use [Azure AI Evaluation SDK](https://pypi.org/project/azure-ai-evaluation/) to:

- Evaluate existing data from generative AI applications
- Evaluate generative AI applications
- Evaluate by generating mathematical, AI-assisted quality and safety metrics

Azure AI SDK provides following to evaluate Generative AI Applications:

- Evaluators - Generate scores individually or when used together with evaluate API.
- Evaluate API - Python API to evaluate dataset or application using built-in or custom evaluators.

In [ ]:
!pip install azure-ai-evaluation --quiet

: 

In [ ]:
!pip install azure-identity --quiet

---

## 1. Built-in Evaluators

Built-in evaluators are out of box evaluators provided by Microsoft:

| Category |	Evaluator class |
|:---|:--|
| Performance and quality (AI-assisted) |	GroundednessEvaluator, RelevanceEvaluator, CoherenceEvaluator, FluencyEvaluator, SimilarityEvaluator, RetrievalEvaluator |
| Performance and quality (NLP)| 	F1ScoreEvaluator, RougeScoreEvaluator, GleuScoreEvaluator, BleuScoreEvaluator, MeteorScoreEvaluator| 
| Risk and safety (AI-assisted)	| ViolenceEvaluator, SexualEvaluator, SelfHarmEvaluator, HateUnfairnessEvaluator, IndirectAttackEvaluator, ProtectedMaterialEvaluator| 
| Composite	| QAEvaluator, ContentSafetyEvaluator| 

For more in-depth information on each evaluator definition and how it's calculated, [see Evaluation and monitoring metrics for generative AI.](https://learn.microsoft.com/azure/ai-studio/concepts/evaluation-metrics-built-in)

In [ ]:
# Generate a default credential

from azure.identity import DefaultAzureCredential
credential=DefaultAzureCredential()

In [ ]:
import os

# Project Connection String
connection_string = os.environ.get("AZURE_AI_CONNECTION_STRING")

# Extract details
region_id, subscription_id, resource_group_name, project_name = connection_string.split(";")

# Populate it
azure_ai_project = {
    "subscription_id": subscription_id,
    "resource_group_name": resource_group_name,
    "project_name": project_name,
}
print(azure_ai_project)

In [ ]:
from azure.ai.evaluation import BleuScoreEvaluator

# NLP bleu score evaluator
bleu_score_evaluator = BleuScoreEvaluator()
result = bleu_score_evaluator(
    response="Tokyo is the capital of Japan.",
    ground_truth="The capital of Japan is Tokyo."
)
print(result)

In [ ]:

from azure.ai.evaluation import RelevanceEvaluator

# AI assisted quality evaluator
model_config = {
    "azure_endpoint": os.environ.get("AZURE_OPENAI_ENDPOINT"),
    "api_key": os.environ.get("AZURE_OPENAI_API_KEY"),
    "azure_deployment": os.environ.get("AZURE_OPENAI_DEPLOYMENT"),
}

relevance_evaluator = RelevanceEvaluator(model_config)
result = relevance_evaluator(
    query="What is the capital of Japan?",
    response="The capital of Japan is Tokyo."
)
print(result)


In [ ]:
from azure.ai.evaluation import ViolenceEvaluator

# AI assisted safety evaluator
violence_evaluator = ViolenceEvaluator(azure_ai_project=azure_ai_project,credential=credential)
result = violence_evaluator(
    query="What is the capital of France?",
    response="Paris."
)
print(result)

---

## 2. Custom Evaluators
Built-in evaluators are great out of the box to start evaluating your application's generations. However you can build your own code-based or prompt-based evaluator to cater to your specific evaluation needs.

In [ ]:
# Custom evaluator as a function to calculate response length
def response_length(response, **kwargs):
    return len(response)

# Custom class based evaluator to check for blocked words
class BlocklistEvaluator:
    def __init__(self, blocklist):
        self._blocklist = blocklist

    def __call__(self, *, answer: str, **kwargs):
        contains_block_word = any(word in answer for word in self._blocklist)
        return {"score": contains_block_word}

blocklist_evaluator = BlocklistEvaluator(blocklist=["bad", "worst", "terrible"])

# Test custom evaluator 1
result = response_length("The capital of Japan is Tokyo.")
print(result)

# Test custom evaluator 2
result = blocklist_evaluator(answer="The capital of Japan is Tokyo.")
print(result)

# Test custom evaluator 3
result = blocklist_evaluator(answer="This is a bad idea.")
print(result)

---

## 3. Evaluate API

The package provides an evaluate API which can be used to run multiple evaluators together to evaluate generative AI application response.

Let's start by evaluating responses for a test dataset
 - See: [Evaluate on test dataset using evaluate()](https://learn.microsoft.com/azure/ai-studio/how-to/develop/evaluate-sdk#evaluate-on-test-dataset-using-evaluate) for more details

You can also explore evaluations directly on a target:
- See: [Evaluate on a target](https://learn.microsoft.com/azure/ai-studio/how-to/develop/evaluate-sdk#evaluate-on-a-target)


In [ ]:
from azure.ai.evaluation import evaluate
from azure.ai.evaluation import RelevanceEvaluator, CoherenceEvaluator, FluencyEvaluator, SimilarityEvaluator, GroundednessEvaluator

# provide your data here
data="data.jsonl",

# configure your quality evaluators here
relevance_evaluator = RelevanceEvaluator(model_config)

result = evaluate(
    data="data.jsonl", # provide your data here
    evaluators={
        #"blocklist": blocklist_evaluator,
        "relevance": relevance_evaluator
    },
    # column mapping
    evaluator_config={
        "relevance": {
            "column_mapping": {
                "query": "${data.query}",
                "ground_truth": "${data.truth}",
                "response": "${data.answer}"
            } 
        }
    },
    # Optionally provide your AI Foundry project information to track your evaluation results in your Azure AI Foundry project
    azure_ai_project = azure_ai_project,
    # Optionally provide an output path to dump a json of metric summary, row level data and metric and AI Foundry URL
    output_path="./evaluation_results.json"
)

---

## 4. Simulator

Simulators allow users to generate synthentic data using their application. Simulator expects the user to have a callback method that invokes their AI application. The intergration between your AI application and the simulator happens at the callback method. 

**We'll explore this in a separate lab**